# Create custom indicator in vectorBT
* Integration into code architecture
* Taking Advantage of Finding Unique Parameters

In [13]:
import vectorbt as vbt
import numpy as np
import datetime
vbt.settings.set_theme('dark')


In [15]:

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=7)

btc_price = vbt.YFData.download(['BTC-USD'],
                                interval='1m',
                                start=start_date,
                                end=end_date,
                                missing_index='drop').get('Close')



In [16]:

def custom_indicator(
    close,
    top_1,
    down_1,
    top_2,
    down_2,
    rsi_window_1,
    rsi_window_2
):
    '''
    Custom indicator for plotting signals between different timeframes
    '''
    # Resample data frame
    close = close['Close']
    close_1 = close.resample('1min').last()
    close_2 = close.resample('5min').last()
    # Calculate indicators and add to data frame
    rsi_1 = vbt.RSI.run(close_1, rsi_window_1).rsi
    rsi_1, _ = rsi_1.align(close, broadcast_axis=0,
                           method='ffill', join='right')
    rsi_1 = rsi_1.to_numpy()

    rsi_2 = vbt.RSI.run(close_2, window=rsi_window_2).rsi
    rsi_2, _ = rsi_2.align(close, broadcast_axis=0,
                           method='ffill', join='right')
    rsi_2 = rsi_2.to_numpy()
    # Calculate signals
    signal = np.where(
        (rsi_1 < down_1) & (rsi_2 > down_2),
        1,
        0)
    signal = np.where(
        (rsi_2 > top_1) & (rsi_2 > top_2),
        -1,
        signal)
    # Reutrn signals in numpy anaray
    return signal


# setting up indicator factory
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['Close'],
    param_names=['top_1', 'down_1', 'top_2',
                 'down_2', 'rsi_window_1', 'rsi_window_2'],
    output_names=['value']).from_apply_func(
    custom_indicator,
    top_1=70, down_1=30,
    top_2=70, down_2=30,
    rsi_window_1=14, rsi_window_2=14,
    keep_pd=True
)

# Define strategy using indicator factory
res = ind.run(
    btc_price,
    top_1=np.arange(60, 90, 10),
    down_1=np.arange(10, 40, 10),
    top_2=np.arange(60, 90, 10),
    down_2=np.arange(10, 40, 10),
    rsi_window_1=np.arange(10, 20, 5),
    rsi_window_2=np.arange(10, 20, 5),
    param_product=True
)

# Define trading logic
entries = res.value_crossed_above(0)
exits = res.value_crossed_below(0)

entries_short = res.value_crossed_below(0)
exits_short = res.value_crossed_above(0)

# Calculate Portfolio
pf = vbt.Portfolio.from_signals(
    close=btc_price,
    entries=entries,
    exits=exits,
    short_entries=entries_short,
    short_exits=exits_short,
    fees=0.001
)

# Print the best parameters
returns = pf.total_return()
print(returns.max())
print(returns.idxmax())


0.13712185797980184
(60, 10, 60, 20, 10, 10)


### Analyse parameters of custom indicators to undestand profit zones

In [10]:
fig = returns.vbt.volume(x_level='comb_top_1',
                         y_level='comb_down_1',
                         z_level='comb_rsi_window_1',
                         height=400, width=1700)
fig.show()

### Backtest the best parameters

In [17]:
res = ind.run(
    btc_price,
    top_1=60,
    down_1=10,
    top_2=70,
    down_2=30,
    rsi_window_1=10,
    rsi_window_2=15,
    param_product=True
)
entries = res.value_crossed_above(0)
exits = res.value_crossed_below(0)

entries_short = res.value_crossed_below(0)
exits_short = res.value_crossed_above(0)
pf = vbt.Portfolio.from_signals(
    close=btc_price,
    entries=entries,
    exits=exits,
    short_entries=entries_short,
    short_exits=exits_short,
    fees=0.001
)
pf.stats()

/home/serg/VECTOR/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/home/serg/VECTOR/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/home/serg/VECTOR/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/home/serg/VECTOR/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2022-05-02 14:29:00+00:00
End                           2022-05-09 14:21:00+00:00
Period                                             7244
Start Value                                       100.0
End Value                                    113.501848
Total Return [%]                              13.501848
Benchmark Return [%]                         -15.366971
Max Gross Exposure [%]                            100.0
Total Fees Paid                                3.139325
Max Drawdown [%]                               3.420722
Max Drawdown Duration                            2479.0
Total Trades                                         15
Total Closed Trades                                  14
Total Open Trades                                     1
Open Trade PnL                                -2.082644
Win Rate [%]                                  64.285714
Best Trade [%]                                 7.951171
Worst Trade [%]                               -0

In [18]:
pf.plots().show()